# ☁️ Partie 2 : Collecte Météorologique (API)

## 🎯 Objectif
Interroger une source de données fiable pour obtenir les prévisions à 7 jours. Nous cherchons à identifier les villes offrant les meilleures conditions (Température & Précipitations) pour la semaine à venir.

## 🛠 Choix Techniques
* **OpenWeatherMap One Call 3.0** : Utilisation de cette API spécifique pour avoir une visibilité sur 7 jours (contrairement à l'API standard limitée à 5 jours).
* **Sécurité (`python-dotenv`)** : La clé API n'est pas écrite en dur dans le code mais chargée depuis un fichier `.env` sécurisé et ignoré par Git.
* **Calcul de Score** : Agrégation des données journalières pour trier les villes objectivement.

In [1]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv

# Chargement des variables d'environnement (Sécurité)
load_dotenv()
API_KEY = os.getenv("OPENWEATHER_API_KEY")

if not API_KEY:
    raise ValueError("❌ Erreur : Clé API non trouvée. Vérifiez votre fichier .env")

# Chargement des villes
try:
    df_cities = pd.read_csv('cities_coordinates.csv')
except FileNotFoundError:
    print("❌ Erreur : Le fichier 'cities_coordinates.csv' est manquant. Lancez le notebook 01 d'abord.")

weather_data = []

print("☁️ Démarrage de la collecte Météo (Prévisions 7 jours)...")

for index, row in df_cities.iterrows():
    lat = row['latitude']
    lon = row['longitude']
    city = row['city']
    
    # URL One Call API 3.0
    url = f"https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&exclude=current,minutely,hourly,alerts&units=metric&appid={API_KEY}"
    
    try:
        response = requests.get(url)
        data = response.json()
        
        if "daily" in data:
            # On récupère les 7 prochains jours (index 1 à 7, car 0 est aujourd'hui)
            daily_forecasts = data['daily'][1:8] 
            
            # Calcul des moyennes sur la semaine
            avg_temp = sum(d['temp']['day'] for d in daily_forecasts) / len(daily_forecasts)
            total_rain = sum(d.get('rain', 0) for d in daily_forecasts)
            
            # Récupération de l'ID météo du jour principal (pour savoir s'il y a des nuages/soleil)
            main_weather = daily_forecasts[0]['weather'][0]['main']

            weather_data.append({
                "city_name": city,
                "latitude": lat,
                "longitude": lon,
                "avg_temp_7_days": round(avg_temp, 2),
                "total_rain_7_days": round(total_rain, 2),
                "main_weather": main_weather
            })
            print(f"   ✅ {city} : {round(avg_temp, 1)}°C moy.")
        else:
            print(f"   ⚠️ Pas de données daily pour {city}")
            
    except Exception as e:
        print(f"   ❌ Erreur API pour {city}: {e}")

# Sauvegarde
df_weather = pd.DataFrame(weather_data)
df_weather.to_csv('cities_weather_data_7days.csv', index=False)

print("\n" + "="*30)
print(f"🎉 Terminé ! Données récupérées pour {len(df_weather)} villes.")
print("💾 Sauvegardé dans 'cities_weather_data_7days.csv'")
df_weather.head()

☁️ Démarrage de la collecte Météo (Prévisions 7 jours)...


   ✅ Mont Saint Michel : 8.3°C moy.


   ✅ St Malo : 8.3°C moy.


   ✅ Bayeux : 7.6°C moy.


   ✅ Le Havre : 7.2°C moy.


...

   ✅ Montauban : 11.2°C moy.


   ✅ Biarritz : 10.6°C moy.


   ✅ Bayonne : 10.5°C moy.


   ✅ La Rochelle : 9.2°C moy.

🎉 Terminé ! Données récupérées pour 35 villes.
💾 Sauvegardé dans 'cities_weather_data_7days.csv'


,city_name,latitude,longitude,avg_temp_7_days,total_rain_7_days,main_weather
0,Mont Saint Michel,48.635954,-1.511460,8.33,12.24,Rain
1,St Malo,48.649518,-2.026041,8.26,11.08,Rain
2,Bayeux,49.276462,-0.702474,7.62,14.56,Rain
3,Le Havre,49.493898,0.107973,7.19,13.98,Rain
4,Rouen,49.440459,1.093966,7.22,10.90,Rain
